# [Research Request - 5311 Apportionment Formula #1710](https://github.com/cal-itp/data-analyses/issues/1710)

- Do the past 3 years of 5311 apportionments to regional bodies passing through Caltrans align with the stated formula? If not, how would apportionments change? How would apportionments change if allocations went directly to each 5311-recipient agency?


- Double check work done by Sara.

- Cross reference all available types of census data.

- can we use the census API to read in data to a notebook? AND apple the apportionment formula

Notes from Grant team 5311 apportionment excel sheet
1. Metropolitan Transportation Commission (MTC): 
  - Sonoma, 
  - Napa, 
  - Solano, 
  - Contra Costa, 
  - Alameda, 
  - Marin, 
  - San Francisco, 
  - San Mateo and 
  - Santa Clara
2. Sacramento Area Council of Governments (SACOG): 
  - Yuba, 
  - Sutter, 
  - Yolo and 
  - Sacramento
3. Alpine and Sierra Counties (collectively, 'Minimum Counties') receive a minimum funding amount of $48,000 due to low population (under 7,590)
4. Orange County does not receive Rural funds


In [1]:
import pandas as pd
import requests

## Census Datasets

### API data

In [2]:
API_Data = "https://api.census.gov/data/2020/dec/cd118?get=group(P2)&ucgid=pseudo(0100000US$0400000)"

In [3]:
df = pd.read_json(API_Data)

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,GEO_ID,NAME,P2_001N,P2_001NA,P2_002N,P2_002NA,P2_003N,P2_003NA,P2_004N,P2_004NA,ucgid
1,0400000US01,Alabama,5024279,None,2900880,None,2123399,None,0,None,0400000US01
2,0400000US02,Alaska,733391,None,475967,None,257424,None,0,None,0400000US02
3,0400000US04,Arizona,7151502,None,6385230,None,766272,None,0,None,0400000US04
4,0400000US05,Arkansas,3011524,None,1670677,None,1340847,None,0,None,0400000US05


### State-level Urban and Rural Information for the 2020 Census and 2010 Census
https://www.census.gov/programs-surveys/geography/guidance/geo-areas/urban-rural.html

In [5]:
state_rural_url = "https://www2.census.gov/geo/docs/reference/ua/State_Urban_Rural_Pop_2020_2010.xlsx"

# r = requests.get(state_rural_url)



# with open("state_rural.xlsx", "wb") as f:
#     f.write(r.content)

state_rural_data = pd.read_excel(state_rural_url) # getting error

display(
    state_rural_data.info(),
    state_rural_data.head()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   STATEFP             56 non-null     int64  
 1   STATE ABBREV        56 non-null     object 
 2   STATE NAME          56 non-null     object 
 3   2020 TOTAL POP      56 non-null     int64  
 4   2020 
URBAN POP     56 non-null     int64  
 5   2020 PCT URBAN POP  56 non-null     float64
 6   2020 
RURAL POP     56 non-null     int64  
 7   2020 PCT RURAL POP  56 non-null     float64
 8   2010 
TOTAL POP     56 non-null     int64  
 9   2010 
URBAN POP     56 non-null     int64  
 10  2010 PCT URBAN POP  56 non-null     float64
 11  2010
 RURAL POP     56 non-null     int64  
 12  2010 PCT RURAL POP  56 non-null     float64
dtypes: float64(4), int64(7), object(2)
memory usage: 5.8+ KB


None

,STATEFP,STATE ABBREV,STATE NAME,2020 TOTAL POP,2020 \nURBAN POP,2020 PCT URBAN POP,2020 \nRURAL POP,2020 PCT RURAL POP,2010 \nTOTAL POP,2010 \nURBAN POP,2010 PCT URBAN POP,2010\n RURAL POP,2010 PCT RURAL POP
0,1,AL,Alabama,5024279,2900880,57.737240,2123399,42.262760,4779736,2821804,59.04,1957932,40.96
1,2,AK,Alaska,733391,475967,64.899487,257424,35.100513,710231,468893,66.02,241338,33.98
2,4,AZ,Arizona,7151502,6385230,89.285160,766272,10.714840,6392017,5740659,89.81,651358,10.19
3,5,AR,Arkansas,3011524,1670677,55.476131,1340847,44.523869,2915918,1637589,56.16,1278329,43.84
4,6,CA,California,39538223,37259490,94.236633,2278733,5.763367,37253956,35373606,94.95,1880350,5.05


### County-level Urban and Rural information for the 2020 Census

In [10]:
county_rural_2020_url = "https://www2.census.gov/geo/docs/reference/ua/2020_UA_COUNTY.xlsx"

county_rural_2010_url = "https://www2.census.gov/geo/docs/reference/ua/PctUrbanRural_County.xls"

county_rural_data_2020 = pd.read_excel(county_rural_url)

county_rural_data_2010 = pd.read_excel(county_rural_2010_url)

In [11]:
display(
    county_rural_data_2020.info(),
    county_rural_data_2010.info(),
    county_rural_data_2020.head(),
    county_rural_data_2010.head(),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3234 entries, 0 to 3233
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   STATE          3234 non-null   int64  
 1   COUNTY         3234 non-null   int64  
 2   STATE_NAME     3234 non-null   object 
 3   COUNTY_NAME    3234 non-null   object 
 4   POP_COU        3234 non-null   int64  
 5   HOU_COU        3234 non-null   int64  
 6   ALAND_COU      3234 non-null   int64  
 7   ALAND_Mi²_COU  3234 non-null   float64
 8   POPDEN_COU     3234 non-null   float64
 9   HOUDEN_COU     3234 non-null   float64
 10  TOTALBLOCKS    3234 non-null   int64  
 11  POP_URB        3234 non-null   int64  
 12  POPPCT_URB     3234 non-null   float64
 13  HOU_URB        3234 non-null   int64  
 14  HOUPCT_URB     3234 non-null   float64
 15  ALAND_URB      3234 non-null   int64  
 16  ALAND_Mi²_URB  3234 non-null   float64
 17  ALAND_PCT_URB  3234 non-null   float64
 18  POPDEN_U

None

None

,STATE,COUNTY,STATE_NAME,COUNTY_NAME,POP_COU,HOU_COU,ALAND_COU,ALAND_Mi²_COU,POPDEN_COU,HOUDEN_COU,...,POP_RUR,POPPCT_RUR,HOU_RUR,HOUPCT_RUR,ALAND_RUR,ALAND_Mi²_RUR,ALAND_PCT_RUR,POPDEN_RUR,HOUDEN_RUR,RURALBLOCKS
0,1,1,Alabama,Autauga,58805,24350,1539634184,594.452758,98.922916,40.962044,...,23920,0.406768,9991,0.410308,1483727020,572.867002,0.963688,41.754892,17.440348,991
1,1,3,Alabama,Baldwin,231767,124148,4117656199,1589.827058,145.781265,78.088997,...,87113,0.375865,40740,0.328157,3762600021,1452.739868,0.913772,59.964624,28.043562,3181
2,1,5,Alabama,Barbour,25223,11618,2292160149,885.003034,28.500467,13.127639,...,16627,0.659200,7538,0.648821,2276027730,878.774307,0.992962,18.920672,8.577857,1011
3,1,7,Alabama,Bibb,22293,9002,1612188717,622.466064,35.814001,14.461833,...,22293,1.000000,9002,1.000000,1612188717,622.466064,1.000000,35.814001,14.461833,1090
4,1,9,Alabama,Blount,59134,24622,1670259090,644.887035,91.696680,38.180330,...,53510,0.904894,22337,0.907197,1658933117,640.514076,0.993219,83.542270,34.873551,2207


,STATE,COUNTY,STATENAME,COUNTYNAME,POP_COU,AREA_COU,POP_URBAN,POPPCT_URBAN,AREA_URBAN,AREAPCT_URBAN,...,POP_UC,POPPCT_UC,AREA_UC,AREAPCT_UC,POPDEN_UC,POP_RURAL,POPPCT_RURAL,AREA_RURAL,AREAPCT_RURAL,POPDEN_RURAL
0,1,1,Alabama,Autauga,54571,1539582278,31650,58.00,50882539,3.30,...,0,0.00,0,0.00,NaN,22921,42.00,1488699739,96.70,39.9
1,1,3,Alabama,Baldwin,182265,4117521611,105205,57.72,275001368,6.68,...,41556,22.80,111381848,2.71,966.3,77060,42.28,3842520243,93.32,51.9
2,1,5,Alabama,Barbour,27457,2291818968,8844,32.21,17447656,0.76,...,8844,32.21,17447656,0.76,1312.8,18613,67.79,2274371312,99.24,21.2
3,1,7,Alabama,Bibb,22915,1612480789,7252,31.65,18903236,1.17,...,7252,31.65,18903236,1.17,993.6,15663,68.35,1593577553,98.83,25.5
4,1,9,Alabama,Blount,57322,1669961855,5760,10.05,10874700,0.65,...,5374,9.38,9851721,0.59,1412.8,51562,89.95,1659087155,99.35,80.5


In [14]:
ca_county_2010 = county_rural_data_2010[county_rural_data_2010["STATENAME"]=="California"]

In [15]:
ca_county_2020 = county_rural_data_2020[county_rural_data_2020["STATE_NAME"]=="California"]

In [19]:
ca_county_2010["COUNTYNAME"].nunique() == ca_county_2020["COUNTY_NAME"].nunique()

True

In [27]:
county_2010 = ca_county_2010.groupby("COUNTYNAME").agg(
    {"POP_RURAL":"sum", 
     "POP_COU":"sum", 
     "POPPCT_RURAL":"sum"}).reset_index().rename(columns={"COUNTYNAME":"county", ""})

,COUNTYNAME,POP_RURAL,POP_COU,POPPCT_RURAL
0,Alameda,5869,1510271,0.39
1,Alpine,1175,1175,100.00
2,Amador,23016,38091,60.42
3,Butte,41584,220000,18.90
4,Calaveras,34370,45578,75.41
5,Colusa,6795,21419,31.72
6,Contra Costa,8316,1049025,0.79
7,Del Norte,9634,28610,33.67
8,El Dorado,62827,181058,34.70
9,Fresno,100537,930450,10.81


In [26]:
ca_county_2020.groupby("COUNTY_NAME").agg({"POP_RUR":"sum", "POP_COU":"sum", "POPPCT_RUR":"sum"}).reset_index()

,COUNTY_NAME,POP_RUR,POP_COU,POPPCT_RUR
0,Alameda,8253,1682353,0.004906
1,Alpine,1204,1204,1.000000
2,Amador,28020,40474,0.692296
3,Butte,44478,211632,0.210167
4,Calaveras,37128,45292,0.819747
5,Colusa,9326,21839,0.427034
6,Contra Costa,11661,1165927,0.010001
7,Del Norte,12123,27743,0.436975
8,El Dorado,70112,191185,0.366723
9,Fresno,113625,1008654,0.112650
